# Create Bronze Layer Shortcuts (cusn schema)

This notebook creates the Bronze layer (cusn schema) with shortcuts to both:
- **Batch Historical Data**: 24 parquet tables in ADLSv2 (6 dimensions + 18 facts)
- **Streaming Real-Time Data**: 18 event tables in Eventhouse

The Bronze layer serves as the unified source for the Silver layer transformation.

## Prerequisites
- ADLSv2 storage account with parquet files (exported from datagen)
- Eventhouse with streaming event tables
- Lakehouse workspace permissions

## Architecture
```
ADLSv2 Parquet (24 tables) ──shortcut──> cusn.dim_*, cusn.fact_*
Eventhouse Events (18 tables) ──shortcut──> cusn.receipt_created, etc.
```

In [ ]:
# Configuration Parameters
# Use environment variables or override with defaults for local testing

import os

# ADLSv2 Configuration
ADLS_ACCOUNT = os.environ.get("ADLS_ACCOUNT", "stdretail")
ADLS_CONTAINER = os.environ.get("ADLS_CONTAINER", "supermarket")
ADLS_BASE_PATH = os.environ.get("ADLS_BASE_PATH", "")  # Base path within container (empty for root)

# Eventhouse Configuration  
EVENTHOUSE_URI = os.environ.get("EVENTHOUSE_URI", "<replace-with-eventhouse-uri>")
EVENTHOUSE_DATABASE = os.environ.get("EVENTHOUSE_DATABASE", "kql_retail_db")

# Schema Names
BRONZE_SCHEMA = os.environ.get("BRONZE_SCHEMA", "cusn")

# Deployment Mode: Set REQUIRE_EVENTHOUSE=true to fail if Eventhouse not configured
# This prevents incomplete Bronze layer in production
REQUIRE_EVENTHOUSE = os.environ.get("REQUIRE_EVENTHOUSE", "false").lower() == "true"

# Validate required configuration
if EVENTHOUSE_URI == "<replace-with-eventhouse-uri>":
    if REQUIRE_EVENTHOUSE:
        raise EnvironmentError(
            "CRITICAL: EVENTHOUSE_URI not configured but REQUIRE_EVENTHOUSE=true.\n"
            "Bronze layer will be incomplete without Eventhouse shortcuts.\n"
            "Either:\n"
            "  1. Set EVENTHOUSE_URI environment variable, OR\n"
            "  2. Set REQUIRE_EVENTHOUSE=false to skip Eventhouse shortcuts (dev/test only)"
        )
    else:
        print("⚠️  WARNING: EVENTHOUSE_URI not configured. Eventhouse shortcuts will NOT be created.")
        print("   Bronze layer will only contain batch parquet shortcuts (24 tables).")
        print("   Set EVENTHOUSE_URI environment variable to enable full Bronze layer (42 tables).")
        print("   For production deployment, set REQUIRE_EVENTHOUSE=true to fail on missing config.")
        CREATE_EVENTHOUSE_SHORTCUTS = False
else:
    CREATE_EVENTHOUSE_SHORTCUTS = True

print(f"\nConfiguration loaded:")
print(f"  ADLSv2: abfss://{ADLS_CONTAINER}@{ADLS_ACCOUNT}.dfs.core.windows.net/{ADLS_BASE_PATH}")
print(f"  Eventhouse: {EVENTHOUSE_URI}/{EVENTHOUSE_DATABASE}")
print(f"  Bronze Schema: {BRONZE_SCHEMA}")
print(f"  Require Eventhouse: {REQUIRE_EVENTHOUSE}")
print(f"  Create Eventhouse Shortcuts: {CREATE_EVENTHOUSE_SHORTCUTS}")

## Step 1: Create Bronze Schema

In [ ]:
# Create cusn schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {BRONZE_SCHEMA}")
print(f"✓ Schema '{BRONZE_SCHEMA}' created or already exists")

# Verify schema creation
schemas = spark.sql("SHOW SCHEMAS").collect()
if any(row.namespace == BRONZE_SCHEMA for row in schemas):
    print(f"✓ Verified: Schema '{BRONZE_SCHEMA}' exists")
else:
    raise Exception(f"Failed to create schema '{BRONZE_SCHEMA}'")

## Step 2: Create Shortcuts to ADLSv2 Parquet Tables (Batch Historical Data)

Creates 24 shortcuts to parquet files in ADLSv2:
- 6 Dimension Tables
- 18 Fact Tables

In [ ]:
# Define dimension tables
dimension_tables = [
    "dim_geographies",
    "dim_stores",
    "dim_distribution_centers",
    "dim_trucks",
    "dim_customers",
    "dim_products"
]

# Define fact tables (all 18)
fact_tables = [
    "fact_receipts",
    "fact_receipt_lines",
    "fact_store_inventory_txn",
    "fact_dc_inventory_txn",
    "fact_truck_moves",
    "fact_truck_inventory",
    "fact_foot_traffic",
    "fact_ble_pings",
    "fact_customer_zone_changes",
    "fact_marketing",
    "fact_online_order_headers",
    "fact_online_order_lines",
    "fact_payments",
    "fact_store_ops",
    "fact_stockouts",
    "fact_promotions",
    "fact_promo_lines",
    "fact_reorders"
]

all_parquet_tables = dimension_tables + fact_tables
print(f"Total parquet tables to create shortcuts for: {len(all_parquet_tables)}")
print(f"  Dimensions: {len(dimension_tables)}")
print(f"  Facts: {len(fact_tables)}")

In [ ]:
from notebookutils import mssparkutils

# Helper function to create ADLSv2 shortcut
def create_adls_shortcut(table_name: str, schema: str = BRONZE_SCHEMA) -> bool:
    """
    Create a shortcut to an ADLSv2 parquet table.
    
    Args:
        table_name: Name of the table (e.g., 'dim_stores')
        schema: Target schema name (default: cusn)
        
    Returns:
        True if successful, False otherwise
    """
    try:
        # Construct ADLS path
        adls_path = f"abfss://{ADLS_CONTAINER}@{ADLS_ACCOUNT}.dfs.core.windows.net/{ADLS_BASE_PATH}{table_name}/"
        
        # Create shortcut using mssparkutils
        # Note: This API may vary based on Fabric version
        # Alternative: Use Fabric REST API or Portal for manual creation
        
        # For now, we'll create external tables pointing to ADLS
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {schema}.{table_name}
            USING PARQUET
            LOCATION '{adls_path}'
        """)
        
        print(f"  ✓ {schema}.{table_name} -> {adls_path}")
        return True
        
    except Exception as e:
        print(f"  ✗ Failed to create {schema}.{table_name}: {e}")
        return False

# Create shortcuts for all parquet tables
print(f"\nCreating shortcuts to ADLSv2 parquet tables...\n")
success_count = 0
failed_tables = []

for table in all_parquet_tables:
    if create_adls_shortcut(table):
        success_count += 1
    else:
        failed_tables.append(table)

print(f"\nADLSv2 Shortcut Creation Summary:")
print(f"  Success: {success_count}/{len(all_parquet_tables)}")
if failed_tables:
    print(f"  Failed tables: {', '.join(failed_tables)}")

## Step 3: Create Shortcuts to Eventhouse Event Tables (Streaming Real-Time Data)

Creates 18 shortcuts to streaming event tables in Eventhouse.

In [ ]:
# Define streaming event tables (matches EventType enum)
event_tables = [
    # Transaction Events
    "receipt_created",
    "receipt_line_added",
    "payment_processed",
    
    # Inventory Events
    "inventory_updated",
    "stockout_detected",
    "reorder_triggered",
    
    # Customer Events
    "customer_entered",
    "customer_zone_changed",
    "ble_ping_detected",
    
    # Operational Events
    "truck_arrived",
    "truck_departed",
    "store_opened",
    "store_closed",
    
    # Marketing Events
    "ad_impression",
    "promotion_applied",
    
    # Omnichannel Events
    "online_order_created",
    "online_order_picked",
    "online_order_shipped"
]

print(f"Total event tables to create shortcuts for: {len(event_tables)}")

In [ ]:
def create_eventhouse_shortcut(table_name: str, schema: str = BRONZE_SCHEMA) -> bool:
    """
    Create a shortcut to an Eventhouse KQL table.
    
    NOTE: Eventhouse shortcuts cannot be created programmatically via Spark SQL.
    This function provides instructions for manual creation via Fabric Portal UI.
    
    For programmatic creation, use:
    - Fabric REST API: POST /v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}/shortcuts
    - PowerShell: New-FabricLakehouseShortcut cmdlet
    
    Args:
        table_name: Name of the event table (e.g., 'receipt_created')
        schema: Target schema name (default: cusn)
        
    Returns:
        False (manual creation required)
    """
    print(f"  ⚠ {schema}.{table_name}: Manual creation required")
    print(f"    1. Navigate to Lakehouse in Fabric workspace")
    print(f"    2. Right-click Tables → New shortcut → Eventhouse")
    print(f"    3. Connection: {EVENTHOUSE_URI}")
    print(f"    4. Database: {EVENTHOUSE_DATABASE}")
    print(f"    5. Table: {table_name}")
    print(f"    6. Target location: Tables/{schema}/{table_name}")
    print()
    return False

# Create shortcuts for all event tables
if CREATE_EVENTHOUSE_SHORTCUTS:
    print(f"\nEventhouse shortcuts must be created manually via Fabric Portal UI.\n")
    print(f"INSTRUCTIONS FOR MANUAL CREATION:")
    print(f"=" * 70)
    print(f"1. Open Fabric workspace and navigate to your Lakehouse")
    print(f"2. In Lakehouse Explorer, right-click on 'Tables' → 'New shortcut'")
    print(f"3. Select source: 'Eventhouse'")
    print(f"4. For each of the {len(event_tables)} event tables below:")
    print(f"   - Connection: {EVENTHOUSE_URI}")
    print(f"   - Database: {EVENTHOUSE_DATABASE}")
    print(f"   - Select table name from list")
    print(f"   - Target: Tables/{BRONZE_SCHEMA}/[table_name]")
    print(f"=" * 70)
    print(f"\nEvent tables to create shortcuts for:")
    
    for i, table in enumerate(event_tables, 1):
        print(f"  {i:2d}. {table}")
    
    print(f"\nAfter creating all shortcuts, run the verification cell below to confirm.")
    
    # Mark all as needing manual creation
    event_success_count = 0
    event_failed_tables = event_tables.copy()
else:
    print(f"\nSkipping Eventhouse shortcut creation (EVENTHOUSE_URI not configured)")
    print(f"Set EVENTHOUSE_URI environment variable to enable Eventhouse shortcuts.")
    event_success_count = 0
    event_failed_tables = []

print(f"\nEventhouse Shortcut Summary:")
print(f"  Programmatic creation: Not supported")
print(f"  Manual creation required: {len(event_tables) if CREATE_EVENTHOUSE_SHORTCUTS else 0} tables")

## Step 4: Verify Bronze Layer Shortcuts

In [ ]:
# List all tables in Bronze schema
bronze_tables = spark.sql(f"SHOW TABLES IN {BRONZE_SCHEMA}").collect()

print(f"\nBronze Schema ({BRONZE_SCHEMA}) Tables:")
print(f"  Total: {len(bronze_tables)} tables\n")

# Categorize tables
dim_count = sum(1 for t in bronze_tables if t.tableName.startswith('dim_'))
fact_count = sum(1 for t in bronze_tables if t.tableName.startswith('fact_'))
event_count = len(bronze_tables) - dim_count - fact_count

print(f"  Dimensions: {dim_count} (expected: 6)")
print(f"  Facts: {fact_count} (expected: 18)")
print(f"  Events: {event_count} (expected: 18)")
print(f"\n  Target: 42 tables (6 dims + 18 facts + 18 events)")

if len(bronze_tables) == 42:
    print(f"\n✓ Bronze layer complete with all 42 shortcuts!")
else:
    print(f"\n⚠ Bronze layer has {len(bronze_tables)}/42 tables")
    print(f"  Check for missing tables and create shortcuts manually if needed")

## Step 5: Test Bronze Layer Access

In [ ]:
# Test reading from Bronze shortcuts
print("\nTesting Bronze layer shortcuts...\n")

# Test dimension shortcut
try:
    df_dim = spark.table(f"{BRONZE_SCHEMA}.dim_stores")
    print(f"✓ dim_stores: {df_dim.count()} rows")
    df_dim.printSchema()
except Exception as e:
    print(f"✗ dim_stores failed: {e}")

print("\n" + "="*60 + "\n")

# Test fact shortcut
try:
    df_fact = spark.table(f"{BRONZE_SCHEMA}.fact_receipts")
    print(f"✓ fact_receipts: {df_fact.count()} rows")
    df_fact.printSchema()
except Exception as e:
    print(f"✗ fact_receipts failed: {e}")

print("\n" + "="*60 + "\n")

# Test event shortcut (if available)
try:
    df_event = spark.table(f"{BRONZE_SCHEMA}.receipt_created")
    print(f"✓ receipt_created: {df_event.count()} rows")
    df_event.printSchema()
except Exception as e:
    print(f"⚠ receipt_created: {e}")
    print(f"  (Event shortcuts may need manual creation)")

print("\n" + "="*60)
print("\nBronze layer testing complete!")

## Summary

This notebook has created the Bronze layer (cusn schema) with shortcuts to:

### ADLSv2 Parquet Tables (24 tables)
- 6 Dimension tables: dim_*
- 18 Fact tables: fact_*

### Eventhouse Event Tables (18 tables)
- Transaction events: receipt_created, receipt_line_added, payment_processed
- Inventory events: inventory_updated, stockout_detected, reorder_triggered
- Customer events: customer_entered, customer_zone_changed, ble_ping_detected
- Operational events: truck_arrived, truck_departed, store_opened, store_closed
- Marketing events: ad_impression, promotion_applied
- Omnichannel events: online_order_created, online_order_picked, online_order_shipped

### Next Steps
1. If any shortcuts failed, create them manually via Fabric UI
2. Verify all 42 shortcuts are accessible
3. Run the Silver transformation notebook (02-onelake-to-silver.ipynb)
4. Silver layer will combine batch + streaming data into unified ag.* tables

### Manual Shortcut Creation (if needed)
For Eventhouse shortcuts that couldn't be created programmatically:
1. Navigate to Lakehouse in Fabric workspace
2. Right-click on Tables
3. Select "New shortcut" → "Eventhouse"
4. Enter Eventhouse URI and table name
5. Target schema: cusn